## Лабораторная работа №4: Проведение исследований со случайным лесом


### 1. Выбор начальных условий

Был проведен в ЛР №1.

In [2]:
!pip install scikit-learn numpy pandas matplotlib seaborn

In [3]:
import os
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [4]:
date_fruit_path = "/kaggle/input/date-fruit-datasets"

print("Path to Date Fruit dataset files:", date_fruit_path)

# Проверка содержимого папки
if os.path.exists(date_fruit_path):
    files = os.listdir(date_fruit_path)
    print("Files:", files)
else:
    print("Not found")

Path to Date Fruit dataset files: /kaggle/input/date-fruit-datasets
Files: ['Date_Fruit_Datasets']


In [5]:
# Чтение Excel-файла
date_fruit_data = pd.read_excel(f"{date_fruit_path}/Date_Fruit_Datasets/Date_Fruit_Datasets.xlsx")

# Проверка данных
date_fruit_data.head()

,AREA,PERIMETER,MAJOR_AXIS,MINOR_AXIS,ECCENTRICITY,EQDIASQ,SOLIDITY,CONVEX_AREA,EXTENT,ASPECT_RATIO,...,KurtosisRR,KurtosisRG,KurtosisRB,EntropyRR,EntropyRG,EntropyRB,ALLdaub4RR,ALLdaub4RG,ALLdaub4RB,Class
0,422163,2378.908,837.8484,645.6693,0.6373,733.1539,0.9947,424428,0.7831,1.2976,...,3.2370,2.9574,4.2287,-59191263232,-50714214400,-39922372608,58.7255,54.9554,47.8400,BERHI
1,338136,2085.144,723.8198,595.2073,0.5690,656.1464,0.9974,339014,0.7795,1.2161,...,2.6228,2.6350,3.1704,-34233065472,-37462601728,-31477794816,50.0259,52.8168,47.8315,BERHI
2,526843,2647.394,940.7379,715.3638,0.6494,819.0222,0.9962,528876,0.7657,1.3150,...,3.7516,3.8611,4.7192,-93948354560,-74738221056,-60311207936,65.4772,59.2860,51.9378,BERHI
3,416063,2351.210,827.9804,645.2988,0.6266,727.8378,0.9948,418255,0.7759,1.2831,...,5.0401,8.6136,8.2618,-32074307584,-32060925952,-29575010304,43.3900,44.1259,41.1882,BERHI
4,347562,2160.354,763.9877,582.8359,0.6465,665.2291,0.9908,350797,0.7569,1.3108,...,2.7016,2.9761,4.4146,-39980974080,-35980042240,-25593278464,52.7743,50.9080,42.6666,BERHI


In [6]:
concrete_strength_path = "/kaggle/input/concrete-compressive-strength"
print("Путь до датасета Concrete Compressive Strength:", concrete_strength_path)
if os.path.exists(concrete_strength_path):
    files = os.listdir(concrete_strength_path)
    print("Содержание:", files)
else:
    print("Не найден")

Путь до датасета Concrete Compressive Strength: /kaggle/input/concrete-compressive-strength
Содержание: ['Concrete Compressive Strength.csv']


In [7]:
# Чтение CSV-файла
concrete_data = pd.read_csv(f"{concrete_strength_path}/Concrete Compressive Strength.csv")

# Проверка данных
concrete_data.head()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age (day),Concrete compressive strength
0,540.0,0.0,0.0,162.0,2.5,1040.0,676.0,28,79.986111
1,540.0,0.0,0.0,162.0,2.5,1055.0,676.0,28,61.887366
2,332.5,142.5,0.0,228.0,0.0,932.0,594.0,270,40.269535
3,332.5,142.5,0.0,228.0,0.0,932.0,594.0,365,41.052780
4,198.6,132.4,0.0,192.0,0.0,978.4,825.5,360,44.296075


### 2. Создание бейзлайна и оценка качества

In [8]:
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV
from sklearn.metrics import accuracy_score, f1_score, mean_squared_error, r2_score, make_scorer
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.preprocessing import StandardScaler, MinMaxScaler, LabelEncoder
from sklearn.pipeline import Pipeline


Разделим датасет для классификации на обучающую и тестовую выборки


In [9]:
# Разделение на признаки и целевую переменную
X_class = date_fruit_data.drop(columns=['Class'])
y_class = date_fruit_data['Class']

# Разделение на обучающую и тестовую выборки
X_train_class, X_test_class, y_train_class, y_test_class = train_test_split(
    X_class, y_class, test_size=0.2, random_state=42, stratify=y_class
)

# Преобразование целевой переменной
label_encoder = LabelEncoder()
y_train_class = label_encoder.fit_transform(y_train_class)
y_test_class = label_encoder.transform(y_test_class)

Аналогично разделим датасет для регрессии

In [10]:
# Разделение на признаки и целевую переменную
X_reg = concrete_data.drop(columns=['Concrete compressive strength '])
y_reg = concrete_data['Concrete compressive strength ']

# Разделение на обучающую и тестовую выборки
X_train_reg, X_test_reg, y_train_reg, y_test_reg = train_test_split(
    X_reg, y_reg, test_size=0.2, random_state=42
)

Обучим модели для классификации и регрессии из Sklearn и оценим их

In [11]:
rf_classifier = RandomForestClassifier(n_estimators=100, max_depth=10, random_state=42)
rf_classifier.fit(X_train_class, y_train_class)

y_pred_class = rf_classifier.predict(X_test_class)

accuracy_rf = accuracy_score(y_test_class, y_pred_class)
f1_rf = f1_score(y_test_class, y_pred_class, average='weighted')

print(f"Accuracy: {accuracy_rf:.4f}, F1-Score: {f1_rf:.4f}")

Accuracy: 0.9222, F1-Score: 0.9214


In [12]:
rf_regressor = RandomForestRegressor(n_estimators=100, max_depth=10, random_state=42)
rf_regressor.fit(X_train_reg, y_train_reg)

y_pred_reg = rf_regressor.predict(X_test_reg)

rmse_rf = mean_squared_error(y_test_reg, y_pred_reg, squared=False)
r2_rf = r2_score(y_test_reg, y_pred_reg)

print(f"RMSE: {rmse_rf:.4f}, R²: {r2_rf:.4f}")
     

RMSE: 5.5318, R²: 0.8812


Точность для встроенной в Sklearn модели случайного леса получилась высокой, в частности для регрессора. Для алгоритма KNN результаты были Accuracy: 0.7111, RMSE: 8.2983, что сильно хуже.Попробуем улучшить бейзлайн.

### 3. Улучшение бейзлайна

Будем перебирать гиперпараметры случайного леса, такие, как n_estimators, max_depth, min_samples_split, min_samples_leaf

In [13]:
from sklearn.model_selection import GridSearchCV
import numpy as np
from sklearn.metrics import accuracy_score, f1_score, mean_squared_error, r2_score

# Улучшенные параметры сетки с логической прогрессией
param_grid_classifier = {
    'n_estimators': [50, 100, 150],
    'max_depth': [5, 10, 15, None],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'max_features': ['sqrt', 'log2']
}

param_grid_regressor = {
    'n_estimators': [50, 100, 150],
    'max_depth': [5, 10, 15, None],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'max_features': [0.5, 0.8, 'sqrt']
}

# Подбор гиперпараметров для RandomForestClassifier 
print("=== Random Forest Classifier - Hyperparameter Tuning ===")
grid_search_rf_classifier = GridSearchCV(
    estimator=RandomForestClassifier(random_state=42, n_jobs=-1),
    param_grid=param_grid_classifier,
    scoring='accuracy',
    cv=5,
    verbose=1,
    n_jobs=-1,
    refit=True
)

grid_search_rf_classifier.fit(X_train_class, y_train_class)

# Лучшие параметры и результат на кросс-валидации
best_params_rf_classifier = grid_search_rf_classifier.best_params_
best_cv_score_rf_classifier = grid_search_rf_classifier.best_score_

print(f"Лучшие параметры для Random Forest Classifier: {best_params_rf_classifier}")
print(f"Лучший Accuracy (кросс-валидация): {best_cv_score_rf_classifier:.4f}")

# Оценка на тестовой выборке
best_rf_classifier = grid_search_rf_classifier.best_estimator_
y_pred_class_test = best_rf_classifier.predict(X_test_class)

accuracy_test = accuracy_score(y_test_class, y_pred_class_test)
f1_test = f1_score(y_test_class, y_pred_class_test, average='weighted')

print(f"\n=== Результаты на тестовой выборке ===")
print(f"Test Accuracy: {accuracy_test:.4f}")
print(f"Test F1-Score: {f1_test:.4f}")

# Подбор гиперпараметров для RandomForestRegressor 
print("\n=== Random Forest Regressor - Hyperparameter Tuning ===")
grid_search_rf_regressor = GridSearchCV(
    estimator=RandomForestRegressor(random_state=42),
    param_grid=param_grid_regressor,
    scoring='neg_root_mean_squared_error',
    cv=3,
    verbose=1,
    n_jobs=-1,
    refit=True
)

grid_search_rf_regressor.fit(X_train_reg, y_train_reg)

# Лучшие параметры и результат на кросс-валидации
best_params_rf_regressor = grid_search_rf_regressor.best_params_
best_cv_score_rf_regressor = -grid_search_rf_regressor.best_score_

print(f"Лучшие параметры для Random Forest Regressor: {best_params_rf_regressor}")
print(f"Лучший RMSE (кросс-валидация): {best_cv_score_rf_regressor:.4f}")

# Оценка на тестовой выборке (используем лучшую модель из GridSearchCV)
best_rf_regressor = grid_search_rf_regressor.best_estimator_
y_pred_reg_test = best_rf_regressor.predict(X_test_reg)

rmse_test = np.sqrt(mean_squared_error(y_test_reg, y_pred_reg_test))
r2_test = r2_score(y_test_reg, y_pred_reg_test)
mae_test = np.mean(np.abs(y_test_reg - y_pred_reg_test))

print(f"\n=== Результаты на тестовой выборке ===")
print(f"Test RMSE: {rmse_test:.4f}")
print(f"Test R²: {r2_test:.4f}")

=== Random Forest Classifier - Hyperparameter Tuning ===
Fitting 5 folds for each of 216 candidates, totalling 1080 fits
Лучшие параметры для Random Forest Classifier: {'max_depth': 10, 'max_features': 'sqrt', 'min_samples_leaf': 1, 'min_samples_split': 5, 'n_estimators': 100}
Лучший Accuracy (кросс-валидация): 0.8830

=== Результаты на тестовой выборке ===
Test Accuracy: 0.9389
Test F1-Score: 0.9388

=== Random Forest Regressor - Hyperparameter Tuning ===
Fitting 3 folds for each of 324 candidates, totalling 972 fits
Лучшие параметры для Random Forest Regressor: {'max_depth': None, 'max_features': 0.8, 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 100}
Лучший RMSE (кросс-валидация): 5.2533

=== Результаты на тестовой выборке ===
Test RMSE: 5.4137
Test R²: 0.8863


## 4. Имплементация алгоритма машинного обучения

Напишем собственную реализацию случайного леса для классификации и регрессии, затем обучим модели на тестовых данных и сравним по качеству с реализациями из Sklearn. Будем использовать реализацию решающего дерева из ЛР №3.

In [14]:
import numpy as np
from tqdm import tqdm

class DecisionTreeNode:
    def __init__(self, feature=None, threshold=None, left=None, right=None, value=None):
        self.feature = feature
        self.threshold = threshold
        self.left = left
        self.right = right
        self.value = value

class DecisionTreeClassifierCustom:
    def __init__(self, max_depth=None, min_samples_split=2, min_samples_leaf=1):
        self.max_depth = max_depth
        self.min_samples_split = min_samples_split
        self.min_samples_leaf = min_samples_leaf
        self.root = None

    def _gini(self, y):
        if len(y) == 0:
            return 0
        proportions = np.bincount(y) / len(y)
        return 1 - np.sum(proportions ** 2)

    def _split(self, X, y, feature, threshold):
        mask = X[:, feature] < threshold
        return X[mask], y[mask], X[~mask], y[~mask]

    def _best_split(self, X, y):
        best_score = float('inf')
        best_split = None

        for feature in range(X.shape[1]):
            thresholds = np.unique(X[:, feature])
            for threshold in thresholds:
                X_left, y_left, X_right, y_right = self._split(X, y, feature, threshold)

                if len(y_left) < self.min_samples_leaf or len(y_right) < self.min_samples_leaf:
                    continue

                score = (len(y_left) * self._gini(y_left) + len(y_right) * self._gini(y_right)) / len(y)
                if score < best_score:
                    best_score = score
                    best_split = (feature, threshold)

        return best_split

    def _build_tree(self, X, y, depth):
        n_samples = len(y)
        
        if (depth == self.max_depth or n_samples < self.min_samples_split or 
            len(np.unique(y)) == 1):
            return DecisionTreeNode(value=np.bincount(y).argmax() if n_samples > 0 else 0)

        best_split = self._best_split(X, y)
        if best_split is None:
            return DecisionTreeNode(value=np.bincount(y).argmax())

        feature, threshold = best_split
        X_left, y_left, X_right, y_right = self._split(X, y, feature, threshold)

        if len(y_left) < self.min_samples_leaf or len(y_right) < self.min_samples_leaf:
            return DecisionTreeNode(value=np.bincount(y).argmax())

        left_child = self._build_tree(X_left, y_left, depth + 1)
        right_child = self._build_tree(X_right, y_right, depth + 1)

        return DecisionTreeNode(feature, threshold, left_child, right_child)

    def fit(self, X, y):
        X = np.array(X)
        y = np.array(y)
        self.root = self._build_tree(X, y, 0)

    def _predict(self, x, node):
        if node.value is not None:
            return node.value
        if x[node.feature] < node.threshold:
            return self._predict(x, node.left)
        return self._predict(x, node.right)

    def predict(self, X):
        X = np.array(X)
        return np.array([self._predict(x, self.root) for x in X])

class DecisionTreeRegressorCustom:
    def __init__(self, max_depth=None, min_samples_split=2, min_samples_leaf=1):
        self.max_depth = max_depth
        self.min_samples_split = min_samples_split
        self.min_samples_leaf = min_samples_leaf
        self.root = None

    def _mse(self, y):
        if len(y) == 0:
            return 0
        return np.mean((y - np.mean(y)) ** 2)

    def _split(self, X, y, feature, threshold):
        mask = X[:, feature] < threshold
        return X[mask], y[mask], X[~mask], y[~mask]

    def _best_split(self, X, y):
        best_score = float('inf')
        best_split = None

        for feature in range(X.shape[1]):
            thresholds = np.unique(X[:, feature])
            for threshold in thresholds:
                X_left, y_left, X_right, y_right = self._split(X, y, feature, threshold)

                if len(y_left) < self.min_samples_leaf or len(y_right) < self.min_samples_leaf:
                    continue

                score = (len(y_left) * self._mse(y_left) + len(y_right) * self._mse(y_right)) / len(y)
                if score < best_score:
                    best_score = score
                    best_split = (feature, threshold)

        return best_split

    def _build_tree(self, X, y, depth):
        n_samples = len(y)
        
        if (depth == self.max_depth or n_samples < self.min_samples_split or 
            len(np.unique(y)) == 1):
            return DecisionTreeNode(value=np.mean(y) if n_samples > 0 else 0)

        best_split = self._best_split(X, y)
        if best_split is None:
            return DecisionTreeNode(value=np.mean(y))

        feature, threshold = best_split
        X_left, y_left, X_right, y_right = self._split(X, y, feature, threshold)

        if len(y_left) < self.min_samples_leaf or len(y_right) < self.min_samples_leaf:
            return DecisionTreeNode(value=np.mean(y))

        left_child = self._build_tree(X_left, y_left, depth + 1)
        right_child = self._build_tree(X_right, y_right, depth + 1)

        return DecisionTreeNode(feature, threshold, left_child, right_child)

    def fit(self, X, y):
        X = np.array(X)
        y = np.array(y)
        self.root = self._build_tree(X, y, 0)

    def _predict(self, x, node):
        if node.value is not None:
            return node.value
        if x[node.feature] < node.threshold:
            return self._predict(x, node.left)
        return self._predict(x, node.right)

    def predict(self, X):
        X = np.array(X)
        return np.array([self._predict(x, self.root) for x in X])

class CustomRandomForestClassifier:
    def __init__(self, n_estimators=10, max_depth=None, min_samples_split=2, 
                 min_samples_leaf=1, max_features=None, random_state=42):
        self.n_estimators = n_estimators
        self.max_depth = max_depth
        self.min_samples_split = min_samples_split
        self.min_samples_leaf = min_samples_leaf
        self.max_features = max_features
        self.random_state = random_state
        self.trees = []

    def _bootstrap_sample(self, X, y):
        n_samples = X.shape[0]
        indices = np.random.choice(n_samples, n_samples, replace=True)
        return X[indices], y[indices]

    def fit(self, X, y):
        np.random.seed(self.random_state)
        self.trees = []
        
        for i in tqdm(range(self.n_estimators), desc="Training Random Forest Classifier"):
            X_sample, y_sample = self._bootstrap_sample(np.array(X), np.array(y))
            tree = DecisionTreeClassifierCustom(
                max_depth=self.max_depth,
                min_samples_split=self.min_samples_split,
                min_samples_leaf=self.min_samples_leaf
            )
            tree.fit(X_sample, y_sample)
            self.trees.append(tree)

    def predict(self, X):
        X = np.array(X)
        tree_predictions = np.array([tree.predict(X) for tree in self.trees])
        return np.apply_along_axis(lambda x: np.bincount(x).argmax(), axis=0, arr=tree_predictions)


class CustomRandomForestRegressor:
    def __init__(self, n_estimators=10, max_depth=None, min_samples_split=2, 
                 min_samples_leaf=1, max_features=None, random_state=42):
        self.n_estimators = n_estimators
        self.max_depth = max_depth
        self.min_samples_split = min_samples_split
        self.min_samples_leaf = min_samples_leaf
        self.max_features = max_features
        self.random_state = random_state
        self.trees = []

    def _bootstrap_sample(self, X, y):
        n_samples = X.shape[0]
        indices = np.random.choice(n_samples, n_samples, replace=True)
        return X[indices], y[indices]

    def fit(self, X, y):
        np.random.seed(self.random_state)
        self.trees = []
        
        for i in tqdm(range(self.n_estimators), desc="Training Random Forest Regressor"):
            X_sample, y_sample = self._bootstrap_sample(np.array(X), np.array(y))
            tree = DecisionTreeRegressorCustom(
                max_depth=self.max_depth,
                min_samples_split=self.min_samples_split,
                min_samples_leaf=self.min_samples_leaf
            )
            tree.fit(X_sample, y_sample)
            self.trees.append(tree)

    def predict(self, X):
        X = np.array(X)
        tree_predictions = np.array([tree.predict(X) for tree in self.trees])
        return np.mean(tree_predictions, axis=0)

In [15]:
scaler = StandardScaler()
X_train_class = scaler.fit_transform(X_train_class)
X_test_class = scaler.transform(X_test_class)

custom_rf_classifier = CustomRandomForestClassifier(n_estimators=10, max_depth=5, random_state=42)
custom_rf_classifier.fit(X_train_class, y_train_class)

y_pred_custom_class = custom_rf_classifier.predict(X_test_class)

accuracy_custom = accuracy_score(y_test_class, y_pred_custom_class)
f1_custom = f1_score(y_test_class, y_pred_custom_class, average='weighted')

print(f"Custom Random Forest Classifier - Accuracy: {accuracy_custom:.4f}, F1-Score: {f1_custom:.4f}")

Training Random Forest Classifier: 100%|██████████| 10/10 [00:30<00:00,  3.01s/it]

Custom Random Forest Classifier - Accuracy: 0.8667, F1-Score: 0.8678


In [16]:
scaler = StandardScaler()
X_train_reg = scaler.fit_transform(X_train_reg)
X_test_reg = scaler.transform(X_test_reg)

custom_rf_regressor = CustomRandomForestRegressor(n_estimators=10, max_depth=5, random_state=42)
custom_rf_regressor.fit(X_train_reg, np.array(y_train_reg))

y_pred_custom_reg = custom_rf_regressor.predict(X_test_reg)

rmse_custom = mean_squared_error(y_test_reg, y_pred_custom_reg, squared = False)
r2_custom = r2_score(y_test_reg, y_pred_custom_reg)

print(f"Custom Random Forest Regressor - RMSE: {rmse_custom:.4f}, R²: {r2_custom:.4f}")

Training Random Forest Regressor: 100%|██████████| 10/10 [00:04<00:00,  2.05it/s]

Custom Random Forest Regressor - RMSE: 7.4933, R²: 0.7821


Точность вручную реализованных моделей оказалась похуже, чем у библиотечных, но все равно на приемлемом уровне.